### License
Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: MIT-0

### Overview
This notebook is meant to be an example of how to use the source code for the prompt engineering tracker. The notebook creates a LangChain LLM, and conversationChain to show how the prompt engineering tracker stores relevant information about the inputs and outputs of the chain. 

You will need to update a few things in this notebook to get it to run including:
* the path for the ChainLogCallback
* The question you ask the chain

### Imports and Installs
Install the required libraries from the requirements.txt file

In [ ]:
%pip install -r requirements.txt --quiet

In [ ]:
import re

from langchain.llms import Bedrock
from langchain.tools import tool

from langchain.llms import Bedrock
import boto3
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain_core.runnables import chain
from langchain.prompts import PromptTemplate

from src.chain_log_callback import ChainLogCallback# import UUIDEncoder

### Create callback to log the aspects of the chain
In the cell below, we create the chain log callback so that we can pass it when creating the chain itself.
This callback is how we save information about the chain, such as the prompt template, temperature, question asked, rating, etc.
Note that you can request user inputs in the form of ratings and comments by setting `request_rating` or `request_comments` to true.

The user_name, experiment_name, and path will all determine the path and name of the CSV file. 
The user_name is helpful if you have multiple folks working on a project together - the system adds the user_name into the csv file name.
The experiment_name is helpful so that you can run different types of experiments. Let's say you want to experiment with RAG chains versus with a conversational retrieval chain. You may want those output to different CSV files. Or maybe you start experimenting in another language. You can change the name of the experiment or add a suffix to it (e.g. rag_test_2) to keep this separate.
The path is the path for where to save the CSV files. It may be helpful to create a folder for these to be output so that you dont have to manually git ignore each one. 

Remember that this notebook is meant to be a sample to show you how to use the chain log callback in any application with one of the supported chains.

In [ ]:
callback = ChainLogCallback(output_csv=True, 
                            request_rating=False, 
                            request_comments=False, 
                            user_name="yourUserName", 
                            experiment_name="experiment_name", 
                            path="path/to/where/you/want/the/csv/files", 
                            input_keyword="input",
                            combine_all_actions_into_one_log=True,
                            )

### Create base components of chain
In this section, we create the clients, prompt templates, etc.

In [ ]:
bedrock_client = boto3.client("bedrock-runtime")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

In [ ]:
prompt_template =  """
            Human:
            Your job is to tell me a joke about whatever the human replies. You are extremely funny - you are maybe the funniest AI to ever exist. You have a really dry sense of humor. Your jokes are one line and that is it.
            Try to be original with your jokes
            
            Current conversation:
            <conversation_history>
            {history}
            </conversation_history>

            Here is the human's next reply:
            <human_reply>
            {input}
            </human_reply>

            Assistant:
            """
claude_prompt = PromptTemplate.from_template(prompt_template)

model_id = "anthropic.claude-instant-v1"
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

### Create the LLM
Note that the instantiation of the LLM includes the callback for the LLM. While we have included it here for reference, the code does not do much with the `on_llm_start` method, which is called when the LLM itself starts. Instead, the callback focuses on the chain actions which are more relevant to the chain use cases. 

Here, we use the Bedrock LLM and pass some various inputs to it. 

In [ ]:

llm = Bedrock(
    model_kwargs={"max_tokens_to_sample":350,"temperature":0.1,"top_k":250,"anthropic_version":"bedrock-2023-05-31"},
    model_id=model_id,
    client=bedrock_client,
    verbose=True,
    callbacks=[callback],
)

### Create the chain
When you create the chain, adding the custom callback created early in the notebook to the chain is what enables the logger to run. Without the callback, none of the inputs or outputs will be saved. 

In [ ]:
conversation = ConversationChain(
    llm=llm, verbose=True,
    prompt = claude_prompt,
    memory=ConversationBufferMemory(),
    callbacks=[callback]
)


### Run the chain
Here we run the chain. A CSV should be created / updated with various inputs and outputs of the chain. 

In [ ]:

resp = conversation.predict(input="lions")
print(resp)